In [ ]:
import os
import pandas as pd
import numpy as np

import rasterio

import dask_gateway
import dask.array as da

import refactoring_modules asrm

In [ ]:
# **************************************************************
# ********* SPECIFY ITEMID AND LIDAR YEAR TO MATCH HERE ********

itemid = 'ca_m_3411934_sw_11_060_20200521'
year = 2020

filename = 'SPECTRAL_campus_'+str(year)+'_predictions.tif'

# **************************************************************
# **************************************************************

In [ ]:
cluster = dask_gateway.GatewayCluster()
client = cluster.get_client()
#cluster.adapt(minimum=4, maximum=50)
cluster.scale(30)
client

In [ ]:
raster = rm.rioxr_from_itemid(itemid)

is_veg, water_index, not_veg_index = rm.add_spectral_features(df = rm.raster_as_df(raster.to_numpy(), ['r','g','b','nir']), 
                                                           ndwi_thresh = 0.3, 
                                                           ndvi_thresh = 0.05) 
is_veg.drop('ndwi', axis=1, inplace=True)
is_veg = rm.add_date_features(is_veg, raster.datetime)

da_pixels = da.from_array(np.array(is_veg), chunks=728802)
scene_preds = rfc.predict(da_pixels)
preds = scene_preds.compute()

preds_df = pd.DataFrame(preds, 
                     columns=['is_iceplant'], 
                     index = is_veg.index)
is_iceplant_index = preds_df[preds_df.is_iceplant == 1].index.to_numpy()
non_iceplant_index = preds_df[preds_df.is_iceplant == 0].index.to_numpy()

indices = [non_iceplant_index,
           is_iceplant_index, 
           not_veg_index,
           water_index]
values = [0,
          1,
          2,
          3]

reconstruct = rm.indices_to_image(raster.shape[1], raster.shape[2], indices, values, back_value=100)


if save_raster == True:
    with rasterio.open(
        os.path.join(os.getcwd(),'temp',filename),  # file path
        'w',           # w = write
        driver = 'GTiff', # format
        height = reconstruct.shape[0], 
        width = reconstruct.shape[1],
        count = 1,  # number of raster bands in the dataset
        dtype = rasterio.uint8,
        crs = rast.crs,
        transform = rast.transform,
    ) as dst:
        dst.write(reconstruct.astype(rasterio.uint8), 1)